In [1]:
!pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from includes.data_scrape import Scraper
from includes.df_ops import DataframeOperations
import asyncio

In [3]:
#initialization
profile_url = "https://steamcommunity.com/id/getnymba"
df = pd.DataFrame(columns=['id', 'name', 'country', 'friends'])
scraper = Scraper()
df_ops = DataframeOperations()
max_recursion_depth = 2 #change depth here

In [4]:
#fix here

async def crawl_data(name, steam_id, df, depth=0):
    if depth >= max_recursion_depth:
        return df
    country = await scraper.crawl_country_from_id(steam_id)
    friends = await scraper.crawl_friends_from_id(steam_id)
    # print(f"Depth: {depth} | Steam ID: {steam_id} | Friends: {len(friends)}")
    df_ops.add_data_to_df(df, [steam_id, name, country, friends])
    # if the friend list is not empty, recursively crawl each friend's info
    for friend in friends:
        if friend[0] not in df['id']:
            df = await crawl_data(friend[1], friend[0], df, depth+1)
    return df

In [5]:
async def main():
    my_id = await scraper.crawl_steamid_from_profile(profile_url)
    my_name = "Me"
    await crawl_data("Me", my_id, df, depth=0)

await main()

                  id name        country  \
0  76561198887276207   Me  United States   

                                             friends  
0  [(76561199544356277, six6 ; counter-strike 2 ;...  
                  id                        name        country  \
0  76561198887276207                          Me  United States   
1  76561199544356277  six6 ; counter-strike 2 ;   United States   

                                             friends  
0  [(76561199544356277, six6 ; counter-strike 2 ;...  
1                                                 []  
                  id                        name        country  \
0  76561198887276207                          Me  United States   
1  76561199544356277  six6 ; counter-strike 2 ;   United States   
2  76561198085900397                  ruah ;  ;    Tokyo, Japan   

                                             friends  
0  [(76561199544356277, six6 ; counter-strike 2 ;...  
1                                                 []  


In [9]:
from datetime import datetime

df.to_csv(f'./output/user_data_{datetime.now().strftime("%d%m%y")}.csv', index=False)